In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
# import logging
# import warnings

# logging.basicConfig(level=logging.DEBUG)
# warnings.filterwarnings('ignore')

In [4]:
# import tensorflow as tf

# tf.compat.v1.enable_eager_execution()

In [5]:
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import glowtts
import tensorflow as tf
import numpy as np
import math






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [6]:
config = malaya_speech.config.fastspeech2_config
config['encoder_hidden_size'] = 192
config['encoder_num_hidden_layers'] = 6
config['encoder_attention_head_size'] = 32
config['encoder_intermediate_size'] = 768
config = glowtts.Config(vocab_size = 66, **config)
config_glowtts = glowtts.Config_GlowTTS(malaya_speech.config.glowtts_config)

In [7]:
i = tf.placeholder(tf.int32, [None, None])
i_lengths = tf.placeholder(tf.int32, [None])
mel_outputs = tf.placeholder(tf.float32, [None, None, 80])
mel_lengths = tf.placeholder(tf.int32, [None])

In [8]:
import pickle

with open('dataset-mel.pkl', 'rb') as fopen:
    data, d = pickle.load(fopen)
    
with open('dataset-mel-wav.pkl', 'rb') as fopen:
    wav = pickle.load(fopen)
    
data.keys()

dict_keys(['mel', 'text_ids', 'len_mel', 'len_text_ids', 'stop_token_target', 'f0', 'len_f0', 'energy', 'len_energy', 'g'])

In [9]:
# i = tf.convert_to_tensor(data['text_ids'])
# mel_outputs = tf.convert_to_tensor(data['mel'].astype(np.float32))
# mel_lengths = tf.convert_to_tensor(data['len_mel'][:,0])

In [10]:
model = glowtts.Model(config, config_glowtts)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor




In [11]:
(z, z_m, z_logs, logdet, z_mask), (x_m, x_logs, x_mask), (attn, logw, logw_) = model(i, y = mel_outputs, y_lengths = mel_lengths, training = True)
(z, z_m, z_logs, logdet, z_mask), (x_m, x_logs, x_mask), (attn, logw, logw_)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


((<tf.Tensor 'glowtts/FlowSpecDecoder/mul_4:0' shape=(?, ?, 80) dtype=float32>,
  <tf.Tensor 'glowtts/MatMul_2:0' shape=(?, ?, 80) dtype=float32>,
  <tf.Tensor 'glowtts/MatMul_3:0' shape=(?, ?, 80) dtype=float32>,
  <tf.Tensor 'glowtts/FlowSpecDecoder/add_35:0' shape=<unknown> dtype=float32>,
  <tf.Tensor 'glowtts/transpose:0' shape=(?, ?, 1) dtype=float32>),
 (<tf.Tensor 'glowtts/Encoder/mul:0' shape=(?, ?, 80) dtype=float32>,
  <tf.Tensor 'glowtts/Encoder/zeros_like:0' shape=(?, ?, 80) dtype=float32>,
  <tf.Tensor 'glowtts/Encoder/ExpandDims:0' shape=(?, ?, 1) dtype=float32>),
 (<tf.Tensor 'glowtts/ExpandDims_4:0' shape=(?, 1, ?, ?) dtype=float32>,
  <tf.Tensor 'glowtts/Encoder/ExpandDims_1:0' shape=(?, ?, 1) dtype=float32>,
  <tf.Tensor 'glowtts/mul_8:0' shape=(?, ?, 1) dtype=float32>))

In [12]:
# commons.mle_loss(z, z_m, z_logs, logdet, z_mask)

def mle_loss(z, m, logs, logdet, mask):
    l = tf.reduce_sum(logs) + 0.5 * tf.reduce_sum(tf.math.exp(-2 * logs) * ((z - m)**2))
    l = l - tf.reduce_sum(logdet)
    l = l / tf.reduce_sum(tf.ones_like(z) * mask)
    l = l + 0.5 * math.log(2 * math.pi)
    return l

l_mle = mle_loss(z, z_m, z_logs, logdet, z_mask)
l_mle

<tf.Tensor 'add_1:0' shape=() dtype=float32>

In [13]:
# commons.duration_loss(logw, logw_, x_lengths)

def duration_loss(logw, logw_, lengths):
    # l = torch.sum((logw - logw_)**2) / torch.sum(lengths)
    l = tf.reduce_sum((logw - logw_)**2) / tf.reduce_sum(tf.cast(lengths, tf.float32))
    return l

l_length = duration_loss(logw, logw_, i_lengths)
l_length

<tf.Tensor 'truediv_1:0' shape=() dtype=float32>

In [14]:
loss_g = l_mle + l_length

In [15]:
tf.trainable_variables()

[<tf.Variable 'n_blocks_1/logs:0' shape=(1, 1, 160) dtype=float32_ref>,
 <tf.Variable 'n_blocks_1/bias:0' shape=(1, 1, 160) dtype=float32_ref>,
 <tf.Variable 'n_blocks_1/w_init:0' shape=(4, 4) dtype=float32_ref>,
 <tf.Variable 'n_blocks_2/logs:0' shape=(1, 1, 160) dtype=float32_ref>,
 <tf.Variable 'n_blocks_2/bias:0' shape=(1, 1, 160) dtype=float32_ref>,
 <tf.Variable 'n_blocks_2/w_init:0' shape=(4, 4) dtype=float32_ref>,
 <tf.Variable 'n_blocks_3/logs:0' shape=(1, 1, 160) dtype=float32_ref>,
 <tf.Variable 'n_blocks_3/bias:0' shape=(1, 1, 160) dtype=float32_ref>,
 <tf.Variable 'n_blocks_3/w_init:0' shape=(4, 4) dtype=float32_ref>,
 <tf.Variable 'n_blocks_4/logs:0' shape=(1, 1, 160) dtype=float32_ref>,
 <tf.Variable 'n_blocks_4/bias:0' shape=(1, 1, 160) dtype=float32_ref>,
 <tf.Variable 'n_blocks_4/w_init:0' shape=(4, 4) dtype=float32_ref>,
 <tf.Variable 'n_blocks_5/logs:0' shape=(1, 1, 160) dtype=float32_ref>,
 <tf.Variable 'n_blocks_5/bias:0' shape=(1, 1, 160) dtype=float32_ref>,
 <tf

In [16]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate = 1e-3).minimize(loss_g)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [17]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [18]:
o_ = sess.run([l_mle, l_length], feed_dict = {i: data['text_ids'],
                                   i_lengths: data['len_text_ids'][:,0],
                        mel_outputs: data['mel'],
                        mel_lengths: data['len_mel'][:,0]})
o_

[2.0184827, 6.1939063]

In [19]:
o_ = sess.run([attn, logw, logw_, x_mask], feed_dict = {i: data['text_ids'],
                                   i_lengths: data['len_text_ids'][:,0],
                        mel_outputs: data['mel'],
                        mel_lengths: data['len_mel'][:,0]})

In [20]:
for k in range(300):
    print(k, sess.run([optimizer, l_mle, l_length], feed_dict = {i: data['text_ids'],
                                       i_lengths: data['len_text_ids'][:,0],
                            mel_outputs: data['mel'],
                            mel_lengths: data['len_mel'][:,0]}))

0 [None, 2.020974, 5.101881]
1 [None, 1.8215191, 6.1458282]
2 [None, 1.6109915, 25.574463]
3 [None, 1.2616225, 8.175239]
4 [None, 1.3167629, 3.342362]
5 [None, 1.0505904, 2.742135]
6 [None, 1.0255721, 1.9673386]
7 [None, 0.8809708, 2.1495333]
8 [None, 0.73283076, 2.770985]
9 [None, 0.58802056, 2.4873562]
10 [None, 0.52074957, 2.521972]
11 [None, 0.44607043, 2.413139]
12 [None, 0.34781998, 2.3288474]
13 [None, 0.39179772, 2.0177956]
14 [None, 0.32204807, 2.8698986]
15 [None, 0.3291157, 2.0072174]
16 [None, 0.25190902, 1.8396325]
17 [None, 0.18589652, 1.4476744]
18 [None, 0.11442518, 1.6687226]
19 [None, 0.06573105, 1.6819549]
20 [None, 0.005377114, 1.8619066]
21 [None, -0.042809963, 1.6818018]
22 [None, -0.098561764, 1.501252]
23 [None, -0.13523245, 1.7392095]
24 [None, -0.17615461, 1.9509983]
25 [None, -0.22345269, 1.0825864]
26 [None, -0.25736427, 1.9722627]
27 [None, -0.3141129, 1.3108209]
28 [None, -0.3137287, 1.0718895]
29 [None, -0.36799967, 1.3488059]
30 [None, -0.37551618, 1.001

245 [None, -4.836938, 0.95554817]
246 [None, -4.8665957, 1.5332222]
247 [None, -4.950854, 0.7384904]
248 [None, -5.0277057, 1.0117484]
249 [None, -5.1204906, 0.9755764]
250 [None, -5.180764, 0.8549646]
251 [None, -4.879859, 1.2436472]
252 [None, -4.080675, 0.9520675]
253 [None, -3.785046, 0.66129327]
254 [None, -3.7804804, 0.6409825]
255 [None, -3.740303, 1.2425197]
256 [None, -3.7636976, 0.7976656]
257 [None, -3.8248048, 0.5369543]
258 [None, -3.9023972, 1.1952815]
259 [None, -3.9847183, 0.71596694]
260 [None, -4.077966, 1.0913795]
261 [None, -4.1599708, 0.90527385]
262 [None, -4.257437, 0.93242574]
263 [None, -4.3609467, 1.1580611]
264 [None, -4.4582744, 0.9625981]
265 [None, -4.5526466, 0.85902333]
266 [None, -4.639589, 1.0597713]
267 [None, -4.7221665, 1.0410504]
268 [None, -4.8071613, 0.7950747]
269 [None, -4.904931, 0.8556194]
270 [None, -4.965755, 0.8866437]
271 [None, -5.001454, 1.1506605]
272 [None, -5.063546, 1.1223205]
273 [None, -5.048214, 1.2009779]
274 [None, -5.1931033, 

In [21]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'test/model.ckpt')

'test/model.ckpt'

In [22]:
# !ls -lh test
# !rm -rf test